In [1]:
# Nativos
import random as rn
import os
import sys
import gc

#calculo
import numpy as np
import pandas as pd
import scipy as sp

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)

from utils import *
from graphs import *

SEED = 29082013
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

subfolder = "data"
os.listdir(subfolder)

['CONTENT_CATEGORY.csv',
 '.ipynb_checkpoints',
 'device_data.csv',
 'sampleSubmission.csv',
 'SITE_ID.csv',
 'PAGE.csv',
 'CONTENT_CATEGORY_TOP.csv',
 'conversiones.csv',
 'CONTENT_CATEGORY_BOTTOM.csv',
 'pageviews.csv']

In [2]:
convertion = pd.read_csv(
    "{}/conversiones.csv".format(subfolder)
).drop_duplicates()[['USER_ID','mes']].sort_values(['USER_ID','mes']).reset_index(drop=True)

convertion.head()

,USER_ID,mes
0,13.0,8.0
1,22.0,12.0
2,40.0,1.0
3,57.0,6.0
4,92.0,3.0


In [3]:
for col in convertion.columns:
    convertion[col] = convertion[col].astype(int)

display(convertion.dtypes)
convertion.head()

USER_ID    int64
mes        int64
dtype: object

,USER_ID,mes
0,13,8
1,22,12
2,40,1
3,57,6
4,92,3


In [4]:
convertion['USER_ID'].unique().shape

(1086,)

In [ ]:
for user in convertion['USER_ID'].unique():
    user_conv = convertion[convertion['USER_ID'] == user]
    print(user_conv.shape, end=' ')

(1, 2) (1, 2) (1, 2) (1, 2) (2, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (2, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (2, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (3, 2) (2, 2) (1, 2) (1, 2) (2, 2) (1, 2) (2, 2) (4, 2) (2, 2) (1, 2) (1, 2) (1, 2) (1, 2) (3, 2) (4, 2) (2, 2) (2, 2) (1, 2) (1, 2) (1, 2) (2, 2) (1, 2) (1, 2) (1, 2) (3, 2) (2, 2) (1, 2) (2, 2) (1, 2) (4, 2) (1, 2) (1, 2) (2, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (2, 2) (3, 2) (1, 2) (1, 2) (1, 2) (4, 2) (1, 2) (2, 2) (1, 2) (1, 2) (2, 2) (1, 2) (1, 2) (2, 2) (1, 2) (1, 2) (2, 2) (1, 2) (1, 2) (1, 2) (1, 2) (2, 2) (1, 2) (1, 2) (3, 2) (1, 2) (1, 2) (2, 2) (2, 2) (1, 2) (3, 2) (1, 2) (1, 2) (1, 2) (1, 2) (2, 2) (2, 2) (3, 2) (2, 2) (2, 2) (1, 2) (1, 2) (1, 2) (2, 2) (1, 2) (1, 2) (1, 2) (1, 2) (2, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (2, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (1, 2) (4, 2) (1, 2) (4, 2) (1, 2) (3, 2) (2, 2)

# Se concluye que no hay suficiente informacion historica para realizar un serie de tiempo por usuario, por lo tanto procedemos a centrarno en comportamientos.

In [ ]:
def get_trimestre(val):
    if val <= 3:
        return 1
    elif val <= 6:
        return 2
    elif val <= 9:
        return 3
    elif val <= 12:
        return 4
    return 0

In [ ]:
data = pd.read_csv(
    "{}/pageviews.csv".format(subfolder),
    parse_dates=["FEC_EVENT"],
    usecols=['USER_ID', 'FEC_EVENT', 'PAGE']
).sort_values(
    ['USER_ID', 'FEC_EVENT']
)

data['month'] = data['FEC_EVENT'].dt.month
data['trimestre'] = data['month'].apply(get_trimestre)
data['date_user'] = data['FEC_EVENT'].dt.date
data.head(10)

,FEC_EVENT,PAGE,USER_ID,month,trimestre,date_user
284,2018-01-01 09:56:47,14,0,1,1,2018-01-01
285,2018-01-01 09:57:19,14,0,1,1,2018-01-01
286,2018-01-01 09:57:48,12,0,1,1,2018-01-01
287,2018-01-01 09:57:48,10,0,1,1,2018-01-01
288,2018-01-01 09:57:49,11,0,1,1,2018-01-01
289,2018-01-01 10:02:09,11,0,1,1,2018-01-01
290,2018-01-01 10:02:09,12,0,1,1,2018-01-01
291,2018-01-01 10:02:09,10,0,1,1,2018-01-01
292,2018-01-01 10:07:12,11,0,1,1,2018-01-01
293,2018-01-01 10:07:12,12,0,1,1,2018-01-01


In [ ]:
def get_session(user, datex):
    sesion = data[
        (data['date_user'] == datex) & (data['USER_ID'] == user)
    ]
    return ' '.join(list(
        sesion['PAGE'].astype(str)
    ))

In [ ]:
data_group = data.groupby(by=['USER_ID', 'date_user']).agg({'month': max, 'trimestre': max}).reset_index()

data_group['sesion'] = data[['USER_ID', 'date_user']].apply(lambda _: get_session(_.USER_ID, _.date_user), axis=1)
data_group.head()

In [ ]:
data_group.dtypes